In [ ]:
import cv2
import json
import re
import subprocess

from datetime import datetime
from multiprocessing import Process, JoinableQueue as Queue
from os import listdir, path
from queue import Empty as QueueEmptyException

CAMERA_DB_PATH = "./metadata/cameras.json"

VIDEO_PATH = "/home/vmubuntu/OitoDeJaneiro/vids/0801-500"
DB_PATH = "./metadata/videos.json"

In [ ]:
with open(DB_PATH, "r") as f:
  video_data = json.load(f)

with open(CAMERA_DB_PATH, "r") as f:
  camera_data = json.load(f)

In [ ]:
DATETIME_PATTERN = r'([0-9]{1,2})[-/]([0-9]{1,2})[-/]([0-9]{2,4}) ([0-9]{1,2}):([0-9]{1,2}):([0-9]{1,2})'
DATETIME_FORMAT = '%d%m%Y%H%M%S%z'

def string_to_epoch(datetime_string):
  matches = list(re.search(DATETIME_PATTERN, datetime_string).groups())
  matches = [('00'+m)[-2:] for m in matches]
  matches[2] = ('20'+matches[2])[-4:]
  with_utc_offset = "".join(matches) + "-0300"
  dt = datetime.strptime(with_utc_offset, DATETIME_FORMAT)
  return int(dt.timestamp())

In [ ]:
# TODO: implement this
def ocr(img):
  # TODO: crop img
  return "08-01-2023 15:30:20"

In [ ]:
class Stamp:
  def __init__(self, seconds, timestamp):
    self.timestamp = timestamp
    self.seconds = seconds
  def get_pair(self):
    return [self.timestamp, self.seconds]

def stamp_center(vid, stamp_0, stamp_1):
  diff_seconds = stamp_1.seconds - stamp_0.seconds
  diff_timestamp = stamp_1.timestamp - stamp_0.timestamp

  if (diff_seconds) > 1 and abs(diff_seconds - diff_timestamp) > 1:
    center_seconds = (stamp_1.seconds + stamp_0.seconds) / 2 + stamp_0.seconds
    center_frame = center_seconds * vid.get(cv2.CAP_PROP_FPS)

    vid.set(cv2.CAP_PROP_POS_FRAMES, center_frame)
    _, frame_img = vid.read()

    dts = ocr(frame_img)
    center_timestamp = string_to_epoch(dts)
    stamp_c = Stamp(center_seconds, center_timestamp)

    left_center = [] # stamp_center(vid, stamp_0, stamp_c)
    right_center = [] # stamp_center(vid, stamp_c, stamp_1)

    return left_center + [stamp_c] + right_center
  else:
    return []

In [ ]:
to_process = Queue()

for io_dir in camera_data.keys():
  input_dir_path = path.join(VIDEO_PATH, io_dir)
  input_files = sorted([f for f in listdir(input_dir_path) if f.endswith("mp4")])

  for io_file in input_files:
    input_file_path = path.join(input_dir_path, io_file)

    if io_file not in video_data:
      video_data[io_file] = {
        "name": io_file,
        "camera": io_dir,
      }

    file_data = video_data[io_file]
    vid = None
    if not ("length_seconds" in file_data and "length_frames" in file_data):
      if vid is None:
        vid = cv2.VideoCapture(input_file_path)

      fps = vid.get(cv2.CAP_PROP_FPS)
      length_frames = vid.get(cv2.CAP_PROP_FRAME_COUNT)
      file_data["length_frames"] = length_frames

      vid.set(cv2.CAP_PROP_POS_AVI_RATIO, 1)
      length_seconds_seek = vid.get(cv2.CAP_PROP_POS_MSEC) // 1000
      length_seconds_fps = length_frames // fps

      if abs(length_seconds_fps - length_seconds_seek) > 2:
        print("ERROR: video length", io_file, length_seconds_fps, length_seconds_seek)
      else:
        file_data["length_seconds"] = length_seconds_fps

    if not ("time_start" in file_data and "time_end" in file_data):
      if vid is None:
        vid = cv2.VideoCapture(input_file_path)

      last_frame = vid.get(cv2.CAP_PROP_FRAME_COUNT) - 1

      vid.set(cv2.CAP_PROP_POS_FRAMES, 0)
      _, frame_0 = vid.read()
      dts_0 = ocr(frame_0)
      frame_0_timestamp = string_to_epoch(dts_0)
      stamp_0 = Stamp(0, frame_0_timestamp)

      vid.set(cv2.CAP_PROP_POS_FRAMES, last_frame)
      _, frame_n = vid.read()
      dts_n = ocr(frame_n)
      frame_n_timestamp = string_to_epoch(dts_n)
      stamp_n = Stamp(length_seconds_fps, frame_n_timestamp)

      stamps_c = [] # stamp_center(vid, stamp_0, stamp_n)
      seek = [stamp_0] + stamps_c + [stamp_n]

      file_data["time_start"] = frame_0_timestamp
      file_data["time_end"] = frame_n_timestamp
      file_data["continuous"] = abs((frame_n_timestamp - frame_0_timestamp) - length_seconds_fps) < 1
      file_data["seek"] = [s.get_pair() for s in seek]

    if vid is not None:
      vid.release()

In [ ]:
with open(DB_PATH, "w") as f:
  json.dump(video_data, f, indent=2)